In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import VertexAI
from langchain.chains import LLMChain
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession
import re
import time
from transformers import pipeline

/home/jupyter/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# ***Langchain formatted with Summary Structure & Input Structure***

In [2]:
# Initialize Vertex AI
vertexai.init(project='adsp-capstone-once-upon', location = 'us-central1')
# Initialize the LLM
llm = VertexAI(model_name="gemini-1.0-pro", max_output_tokens=8192, max_tokens = 32000)
# Initialize the text summarization pipeline
summarizer = pipeline("summarization")

# Define prompt template for the summary
summary_prompt_template = PromptTemplate(
    input_variables=["input_prompt", "user_age"],
    template="""
    Generate a complete summary for a 3-chapter story that is engaging for a {user_age}-year-old child. The story should be about: {input_prompt}
    Your response must include:
    1. A brief introduction to the story
    2. Chapter 1 summary (at least 100 words)
    3. Chapter 2 summary (at least 100 words)
    4. Chapter 3 summary (at least 100 words)
    Format your response as follows:
    Story Introduction: [Your introduction here]
    Chapter 1: [Title]
    [Chapter 1 summary]
    Chapter 2: [Title]
    [Chapter 2 summary]
    Chapter 3: [Title]
    [Chapter 3 summary]
    Ensure each chapter summary is complete and shows a clear progression of the story.
    """
)

# Create LLMChain for summary generation
summary_chain = LLMChain(llm=llm, prompt=summary_prompt_template)

def generate_summary(input_prompt, user_age):
    while True:
        summary = summary_chain.run(input_prompt=input_prompt, user_age=user_age)
        print("Complete Story Summary:")
        print(summary)
        return summary

# Define updated prompt template for chapter generation
chapter_prompt_template = PromptTemplate(
    input_variables=["chapter_title", "previous_summary", "chapter_summary", "characters", "chapter_number", "user_age"],
    template=(
        "This is the outline for the chapter: Title: {chapter_title}.\n"
        "Summary of previous chapters: {previous_summary}.\n"
        "Summary for this chapter: {chapter_summary}.\n"
        "Characters and their associated personalities and behaviors so far: {characters}.\n"
        "Based on this outline, generate a LENGTHY chapter {chapter_number} of AT LEAST 1000 WORD TOKENS. "
        "The story should be written in PARAGRAPHS (not bullet points) and fit for a {user_age}-year-old kid in English. "
        "Please ensure the chapter has AT LEAST 1000 words and does not include any additional notes. "
        "Overall, the total number of tokens should be less than 8000 tokens (all characters included)."
    )
)

# Create LLMChain for chapter generation
chapter_chain = LLMChain(llm=llm, prompt=chapter_prompt_template)

def generate_chapter(chapter_title, previous_summary, chapter_summary, characters, chapter_number, user_age):
    chapter = chapter_chain.run(
        chapter_title=chapter_title,
        previous_summary=previous_summary,
        chapter_summary=chapter_summary,
        characters=characters,
        chapter_number=chapter_number,
        user_age=user_age
    )
    print(f"\nChapter {chapter_number} Generated:")
    print(chapter)
    return chapter

# Add this new prompt template for character extraction
character_extraction_prompt = PromptTemplate(
    input_variables=["story_text"],
    template="""
    Extract the main characters from the following story text, along with a brief description of their personalities and behaviors:

    {story_text}

    Format your response as a list of characters, each with a short description:
    - Character Name: Brief description of personality and behavior
    """
)

# Create LLMChain for character extraction
character_extraction_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)

# Add character extraction function
# Update the extract_characters function
def extract_characters(input_story):
    characters = character_extraction_chain.run(story_text=input_story)
    return characters

def generate_story_in_chapters(summary_object, user_age):
    start_time = time.time()
    
    chapters = re.split(r'Chapter \d+:', summary_object)[1:]
    if len(chapters) != 3:
        raise ValueError("The summary does not contain exactly 3 chapters.")
    
    chapter_structures = [f"Chapter {i+1}:{chapter}" for i, chapter in enumerate(chapters)]
    
    # Generate Chapter 1
    chapter1 = generate_chapter(chapter_structures[0].split(':')[1].strip(), "", chapters[0], "", 1, user_age)
    characters = extract_characters(chapter1)
    chapter1_summary = summarizer(chapter1, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    
    # Generate Chapter 2
    chapter2 = generate_chapter(chapter_structures[1].split(':')[1].strip(), chapter1_summary, chapters[1], characters, 2, user_age)
    chapter2_summary = summarizer(chapter2, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    
    # Generate Chapter 3
    previous_summary = f"Chapter 1: {chapter1_summary}\nChapter 2: {chapter2_summary}"
    chapter3 = generate_chapter(chapter_structures[2].split(':')[1].strip(), previous_summary, chapters[2], characters, 3, user_age)
    
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total time taken: {total_time} seconds")
    
    return chapter1, chapter2, chapter3

def chat_loop():
    while True:
        user_age = int(input("Enter the age of the child (e.g., 5): "))
        input_from_user = input("What would you like the story to be about? ")
        
        summary_object = generate_summary(input_from_user, user_age)
        user_response = input("Do you want to continue with this summary? (YES/NO): ")
        
        while user_response.upper() == "NO":
            question = input("Do you want a new story or make changes to this storyline? (NEW STORY/MODIFY): ")
            if question.upper() == "MODIFY":          
                new_prompt = input("Please provide the changes you want to see in this story: ")
                summary_prompt = f"Make the following changes to your story summary, ensuring you maintain complete summaries for all three chapters: {new_prompt}"
                summary_object = summary_chain.run(input_prompt=summary_prompt, user_age=user_age)
                print("\nUpdated Complete Story Summary:")
                print(summary_object)
            elif question.upper() == "NEW STORY":
                input_from_user = input("Let's build you a new story! What would you like to hear a story about? ")
                summary_object = generate_summary(input_from_user, user_age)
            else:
                print("Invalid option. Please choose NEW STORY or MODIFY.")
                continue
            
            user_response = input("Do you want to continue with this summary? (YES/NO): ")
        
        if user_response.upper() == "YES":
            try:
                generated_chapters = generate_story_in_chapters(summary_object, user_age)
                print("\nFull story generated successfully!")
            except Exception as e:
                print(f"An error occurred: {e}")
                print("Let's try again with a new story idea.")
                continue
        
        continue_response = input("Do you want to create another story? (YES/NO): ")
        if continue_response.upper() != "YES":
            print("Thank you for using the story generator. Goodbye!")
            break

if __name__ == "__main__":
    chat_loop()

/home/jupyter/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `VertexAI` was deprecated in LangChain 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Enter the age of the child (e.g., 5):  7
What would you like the story to be about?  A story about bees


/home/jupyter/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Complete Story Summary:
## Story Introduction:

Buzz, a curious young bee, dreams of exploring the world beyond the hive. He longs to follow the older bees on their pollen-collecting adventures, but his mother warns him of the dangers that lurk outside. One sunny morning, Buzz's curiosity gets the better of him, and he embarks on a secret journey, leaving the safety of his home behind.

## Chapter 1: The Escape

Buzz sneaks out of the hive while everyone is busy. He follows a trail of colorful flowers, buzzing with excitement and a little bit of fear. He encounters a friendly ladybug who warns him about the dangers of a nearby spider's web. Buzz, remembering his mother's words, carefully avoids the web and continues his journey.

## Chapter 2: The Lost Bee

As the sun begins to set, Buzz realizes he's lost. He's surrounded by unfamiliar flowers and doesn't know how to get back to the hive. He starts to feel scared and alone. Just when he's about to give up, he sees a faint light in the

Do you want to continue with this summary? (YES/NO):  Yes



Chapter 1 Generated:
## Chapter 1: The Escape

The sun peeked over the horizon, casting a warm glow on the bustling beehive. Inside, the bees were busy as ever, collecting pollen, making honey, and tending to the young. But among them, one little bee named Buzz felt restless. He yearned for adventure, for something beyond the familiar walls of his hive.

Buzz had heard stories from the older bees about the vast world outside, a world filled with colorful flowers, sweet-smelling meadows, and exciting discoveries. His heart buzzed with curiosity, and he longed to explore it all.

One sunny morning, as the other bees were busy with their tasks, Buzz saw his chance. He waited until no one was looking, then quietly slipped out of the hive. He felt a thrill of excitement mixed with a tinge of fear as he flew into the unknown.

The world outside the hive was even more beautiful than Buzz had imagined. The air was fresh and sweet, and the sun warmed his tiny body. He flew through fields of wi

Do you want to create another story? (YES/NO):  No


Thank you for using the story generator. Goodbye!


In [3]:
if __name__ == "__main__":
    chat_loop()

Enter the age of the child (e.g., 5):  6
What would you like the story to be about?  A story about life in a jungle


Complete Story Summary:
## Story Introduction:

Deep in the heart of a lush jungle, where sunlight dappled through the leaves and exotic birds sang their melodies, lived a curious little monkey named Pip. Pip loved swinging from vine to vine, playing with his friends, and exploring every corner of his jungle home. Join Pip on his adventures as he encounters new creatures, learns valuable lessons, and discovers the magic that surrounds him.

## Chapter 1: The Mysterious Footprint

One sunny morning, Pip was swinging through the trees when he spotted something strange on the ground below. It was a giant footprint, unlike anything he had ever seen before. Curiosity bubbling inside him, Pip decided to follow the trail. He scampered through the undergrowth, his eyes peeled for any sign of the mysterious creature. As he followed the tracks, he encountered a wise old toucan named Coco. Coco, known for her vast knowledge of the jungle, explained that the footprint belonged to a rare and elusiv

Do you want to continue with this summary? (YES/NO):  No
Do you want a new story or make changes to this storyline? (NEW STORY/MODIFY):  I want a golden tiger, instead of a golden leopard


Invalid option. Please choose NEW STORY or MODIFY.


Do you want a new story or make changes to this storyline? (NEW STORY/MODIFY):  MODIFY
Please provide the changes you want to see in this story:  I want a golden tiger, instead of a golden leopard



Updated Complete Story Summary:
## Story Introduction:

Deep in the heart of the jungle, where the sun dappled the leaves and the air hummed with the songs of exotic birds, lived a little girl named Maya. Maya was a curious and adventurous soul, always eager to explore the hidden wonders of her world. One day, while venturing deeper into the jungle than she ever had before, Maya stumbled upon a sight that made her heart skip a beat: a magnificent golden tiger, its fur shimmering like the setting sun.

## Chapter 1: The Golden Tiger's Secret

The golden tiger, unlike any Maya had ever seen, was majestic and powerful. Its eyes held a wisdom that seemed to pierce through Maya, and its presence filled her with a sense of awe. As Maya cautiously approached, the tiger lowered its head and let out a soft purr. To Maya's surprise, the tiger spoke, its voice deep and rumbling like thunder. "Do not be afraid, little one," the tiger said. "I am Sunfur, guardian of the jungle. I have been waiting

Do you want to continue with this summary? (YES/NO):  Yes



Chapter 1 Generated:
## Chapter 1: The Golden Tiger's Secret

Maya, a young girl with eyes as bright as the sun and hair as dark as the night, loved exploring the lush green jungle near her village. One sunny afternoon, as she skipped through the tall trees, she stumbled upon a sight that made her heart skip a beat.

There, in a clearing bathed in golden sunlight, stood a magnificent tiger. Unlike any tiger Maya had ever seen, its fur shimmered with a golden hue, making it appear almost magical. Its eyes, the color of molten gold, held a wisdom that seemed to pierce through Maya, and its presence filled her with a sense of awe.

As Maya cautiously approached, the tiger lowered its head and let out a soft purr. To Maya's surprise, the tiger spoke, its voice deep and rumbling like thunder. "Do not be afraid, little one," the tiger said. "I am Sunfur, guardian of the jungle. I have been waiting for you."

Maya's fear melted away, replaced by a sense of wonder. She had never spoken to an 

Do you want to create another story? (YES/NO):  NO


Thank you for using the story generator. Goodbye!
